In [25]:
import json
import pandas as pd
import json
import os
from nltk import sent_tokenize
from tqdm import tqdm_notebook as tqdm

raw_data_path = './raw_data/amazon'
prepared_data_path = './prepared_data/amazon'


In [ ]:
def parse(path):
    with open(path, 'r') as g:
        for l in g:
            yield json.dumps(eval(l))
            
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [37]:
def find_question(question):
    quest = question[0] # if there is no ? in questions, append
    for q in question: #get real question
        if '?' in q:
            return q
    return quest

def process_single(single_dict):
    question = sent_tokenize(single_dict['question'].split('\n')[0])
    question = find_question(question)
    answer = sent_tokenize(single_dict['answer'].split('\n')[0])
    return question, answer

def process_multi(multi_dict):
    question = sent_tokenize(multi_dict['questionText'].split('\n')[0])
    question = find_question(question)
    answer = sent_tokenize(multi_dict['answers'][0]['answerText'].split('\n')[0]) 
    return question, answer

def process(dirpath, single_answer=False):
    files = os.listdir(dirpath)
    questions = []
    answers = []
    for file in tqdm(files):
        if not file.startswith('.'):
            df = getDF(os.path.join(dirpath, file))
            for j in df[0]:
                d = json.loads(j)
                if single_answer:
                    question, answer = process_single(d)
                    if answer:
                        questions.append(question)
                        answers.append(answer[0])  # get first sentence from answer
                else:
                    for k in d['questions']:
                        question, answer = process_multi(k)
                        if answer:
                            questions.append(question)
                            answers.append(answer[0])  
                        
    return questions, answers

In [38]:
import os
def write_sentence_to_file(path, sentences):
    directory = os.path.dirname(os.path.abspath(path))
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(path, 'w') as file_in:
        for sent in sentences:
            file_in.write(sent + '\n')

In [39]:
questions, answers = process(os.path.join(raw_data_path, 'singleanswer'), single_answer=True)
write_sentence_to_file(os.path.join(prepared_data_path, 'singlequestion.txt'), questions)
write_sentence_to_file(os.path.join(prepared_data_path, 'singleanswer.txt'), answers)

In [40]:
multi_questions, multi_answers = process(os.path.join(raw_data_path, 'multianswers'))
write_sentence_to_file(os.path.join(prepared_data_path, 'multiquestion.txt'), multi_questions)
write_sentence_to_file(os.path.join(prepared_data_path, 'multianswers.txt'), multi_answers)